<a href="https://colab.research.google.com/github/0yacinek0/nlp/blob/practice/final/1-%EA%B0%84%EB%8B%A8%ED%95%9C-%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### RNN 모델 이용하여 문장을 순차적으로 입력한 후 최종 hidden vector를 이용하여 문장이 긍정적인지 혹은 부정적인지 판단함


### 필요한 패키지와 랜덤시드 설정

In [ ]:
# !pip install torchtext==0.10.1 
!pip install torch==1.8.0 torchtext==0.9.0
"""
아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.

torchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?
pip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.

https://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils

"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 99.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


'\n아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.\n\ntorchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?\npip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.\n\nhttps://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils\n\n'

In [ ]:
import torch
from torchtext import data

SEED = 1027

""" @
딥러닝에서 초기화를 위해 random num 사용
-> random number 많이 사용
-> 실험마다 다른 값의 연산으로 결과 달라질 수 있음
-> 따라서, 실험을 동일하게 진행하기 위해 동일한 난수 필요 (재생산성 유지)
-> (pytorch) random seed를 구정하기 위한 함수 -> manual_seed 
∴ 랜덤시드 고정 함수 = torch.manual_seed() 
  
https://rabo0313.tistory.com/entry/pytorch-torchmanualseed
"""
torch.manual_seed(SEED)

"""
 deterministic ex. a deterministic view of life
 Deterministic한 알고리즘'만' 사용하게 하기 
 Sets whether PyTorch operations must use “deterministic” algorithms.
 That is, algorithms which, given the same input, and when run on the same software and hardware, always produce the same output
 cudnn에서 수행하는 연산에 대해 적용하기 = True
 *  cudnn(cuda deep neural network) is a GPU-accelerated library of primitives for deep neural networks
"""


torch.backends.cudnn.deterministic = True



구글 드라이브 마운팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd # 현재 경로 확인 

'/content'

In [ ]:
%cd /content/drive/MyDrive/논문/sentimentP/dataset

/content/drive/MyDrive/논문/sentimentP/dataset


In [ ]:
pwd

'/content/drive/MyDrive/논문/sentimentP/dataset'

In [ ]:
ls

Mecab-ko-for-Google-Colab/  ratings_train.txt  train_data.csv
ratings_test.txt            test_data.csv


# 전처리
> Filed 지정 / KoNLPy의 은전한닢(Mecab) tokenizer 이용

In [ ]:
# 코랩 내 다운로드 https://wikidocs.net/94600
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/논문/sentimentP/dataset/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 37.3 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-28 08:03:15--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downl

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
"""
* Pytorch에서 자연어 처리 라이브러리 토치텍스트(torchtext) 제공
* 반드시 필요한 것은 아니나, 직접 구현보다 사용하는 것이 편하기도함
* 토치 텍스트가 제공하는 기능들
1) 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스를 로드합니다.
2) 토큰화(Tokenization) : 문장을 단어 단위로 분리해줍니다.
3) 단어 집합(Vocab) : 단어 집합을 만듭니다.
4) 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑합니다.
5) 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줍니다. 랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수도 있습니다.
6) 배치화(Batching) : 훈련 샘플들의 배치를 만들어줍니다. 이 과정에서 패딩 작업(Padding)도 이루어집니다.
*** 모든 전처리를 해결해주진 X ***
* 검증데이터 분리 작업, 각 샘플에 대해 단어들을 임베딩 벡터로 매핑해주는 작업 (ex. nn.Embedding()을 통해서 lookup table 작업을 해결해야함)

(출처) https://wikidocs.net/60314
"""
import torchtext
from torchtext import data
from torchtext.legacy import data

""""
torchtext.legacy.data에서 제공하는 Field라는 도구를 통해 앞으로 어떤 전처리를 할 것인지 정의 
필드는 어떻게 전처리를 진행할 것인지'만 정의! 실제 train data 전처리 진행 X
"""
TEXT = torchtext.legacy.data.Field(tokenize=mecab.morphs)
LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)

In [ ]:
torch.__version__
torchtext.__version__

'0.9.0'

### torchtext 내 내장된 데이터셋 이용하는게 아니므로 컬럼별 해당 Field 지정

In [ ]:
fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}
# dictionary 형식; {csv 컬럼명 : (데이터 컬럼명, Field이름)}

In [ ]:
"""
* 필드 지정 -> *데이터셋 만들기!* 
* TabularDataset은 데이터 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화 수행 
  (소문자와 같은 기본적인 전처리도 함께 진행)
"""
train_data, test_data = data.TabularDataset.splits(
    path = '.', # 파일이 위치한 경로 
    train = 'train_data.csv',
    test = 'test_data.csv',
    format = 'csv', # 데이터의 포맷 
    fields = fields # 위에서 지정한 필드 지정 
)

In [ ]:
"""
vars() 통해 주어진 인덱스의 샘플 확인
* TabularDataset을 통해 정의한 필드로 구성되어지며 토큰화 진행되었음 
"""
vars(train_data[0]), vars(train_data[1])

({'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'],
  'label': '0'},
 {'text': ['흠',
   '.',
   '..',
   '포스터',
   '보고',
   '초딩',
   '영화',
   '줄',
   '.',
   '...',
   '오버',
   '연기',
   '조차',
   '가볍',
   '지',
   '않',
   '구나'],
  'label': '1'})

In [ ]:
print(f'훈련 데이터 수: {len(train_data)}')
print(f'테스트 데이터 수: {len(test_data)}')

훈련 데이터 수: 149995
테스트 데이터 수: 49997


### 검증데이터 별도 생성

In [ ]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [ ]:
print(f'훈련데이터 수 : {len(train_data)}')
print(f'검증 데이터 수 : {len(valid_data)}')
print(f'테스트 데이터 수 : {len(test_data)}')

훈련데이터 수 : 104996
검증 데이터 수 : 44999
테스트 데이터 수 : 49997


### 단어집합 만들기 *총 단어의 수 확인
> [Q] vocab의 역할? 토큰화 이후, 중복 단어 고려하며 vocab 생성 -> 고유 단어별 정수화해주는 역할?

In [ ]:
"""
* 토큰화 전처리를 끝냈다면 이제 각 단어에 고유한 정수를 맵핑해주는 정수 인코딩 작업이 필요! 
이 전처리 작업을 위해 단어 집합이 필요함!
HOW? 정의한 필드에 build.vocab()도구를 사용하며 단어 집합 생성 
"""
TEXT.build_vocab(train_data)
len(TEXT.vocab)

45983

In [ ]:
# 최소 두 번 이상 등장하는 단어의 갯수
TEXT.build_vocab(train_data, min_freq=2) #  min_freq;단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가
len(TEXT.vocab)

25292

### 25,000개로 끊어 실행

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE) # max_size; 단어집합 최대 크기,
LABEL.build_vocab(train_data)

### unk, pad 토큰이 추가되어 단어의 갯수가 2개 추가됨 

In [ ]:
"""
* 단어들에 일련의 번호를 부여해서 vocabulary를 생성 (문장의 길이를 맞추기 위한 <pad>, <unk> 추가 지정)
* <pad> : padding. 길이를 맞출 때 사용하는 비어있는 토큰 
* <unk> : unknown. 모델이 인식할 수 없는 토큰 ex. 지정한 vocab size 이상인 숫자는 모델이 인식하지 못하고 해당 토큰으로 바꿔 인식
* <sos> : start of sentence(or sequence). 문장의 시작을 알리는 토큰 
* <eos> : end of sequence
(출처) : https://velog.io/@tmddn0311/RNN-tutorial
"""
print(f"TEXT 단어장의 개수 : {len(TEXT.vocab)}")
print(f"LABEL 단어장의 갯수 : {len(LABEL.vocab)}")

TEXT 단어장의 개수 : 25002
LABEL 단어장의 갯수 : 2


### 최빈단어 확인해보기

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 111878), ('이', 51558), ('는', 46843), ('영화', 40324), ('다', 39082), ('고', 33105), ('하', 31175), ('도', 23860), ('의', 23557), ('가', 23373), ('은', 21774), ('에', 21621), ('을', 20954), ('보', 17907), ('한', 17722), ('..', 15987), ('게', 15597), (',', 15423), ('들', 15125), ('!', 13655)]


### 단어와 인덱스 사이 매핑 확인해보기
> [Q] 매핑되었다 = 함께 조합하여 보여준다 (?) 

In [ ]:
""""
List mapping indices to tokens. *indice; index 복수
(출처) : https://pytorch.org/text/stable/vocab.html
"""
print(TEXT.vocab.itos[:10]) 

['<unk>', '<pad>', '.', '이', '는', '영화', '다', '고', '하', '도']


In [ ]:
"""
Dictionary mapping tokens to indices.
string 데이터를 사전의 index로의 변환을 제공함 
"""
print(TEXT.vocab.stoi) # 보통 <unk> 0번, <pad> 1번 부여함
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


### BuketIterator를 이용하여 데이터 생성자 만들기
> [Q1] batch size => 한번에 얼마나 많은 input(text)가 들어갈지 결정 </br>
      ex) 기준은 원본 텍스트 1행?  토큰화 이후 임베딩된 토큰(vocab) 1개 기준? 
          각 행 -> 토큰화 이후 임베딩 -> 텐서로 표현됨 -> HELLO = [[0.1., 0.9., 0.8]] 이런 텐서?로 표현된 입력 텍스트가 기준이 되는 것인지..

> [Q2] Iterator를 변환한다 = Iterator의 개념은 '전체 데이터에 대해 총 Batch의 수'

> [Q3] 배치 내에서 sort한다는 것의 의미는? 


> [Q4] cuda의 개념 * cudnn 란? 



In [ ]:
BATCH_SIZE = 64

"""
gpu에 연산이 할당됨 
* 시스템상 여러개의 gpu가 연결되어 있다면, 그 일부가 다른 gpu로 할당되는 현상이 일어남
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

"""

BucketIterator : Defines an iterator that batches examples of similar lengths together.
classtorchtext.data.BucketIterator(dataset, batch_size, sort_key=None, device=None, batch_size_fn=None, train=True, repeat=False, shuffle=None, sort=None, sort_within_batch=None)

* Pytorch의 dataloader(group examples from the PyTorch Dataset)와 비슷한 역할 (* 다른점? 비슷한 길이의 문장들끼리 batch 만들기 때문에 padding 개수 최소화) 
* 모든 텍스트 작업을 일괄로 처리하고, 단어를 인덱스 숫자로 변환하는 것을 도움
* iterator(반복자) 배열이나 유사한 자료 구조의 내부 요소를 순회하기 위한 객체 
* (더 읽어보기) https://gmihaila.github.io/tutorial_notebooks/pytorchtext_bucketiterator/
"""
# Group similar length text sequences together in batches.
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), # Tuple of train, validation and test batch sizes
    batch_size = BATCH_SIZE,
    device = device, # Device to load batches on.
    sort_key = lambda x: len(x.text), # # Function to use for sorting examples.
    sort_within_batch = False, # Don't Use `sort_key` to sort examples in each batch.

)

AttributeError: ignored

### 생성된 데이터 크기 확인해보기

> [Q] tensor size 이해한게 맞는지..

In [ ]:
"""
파이썬 내장함수 ; iter, next
* iter : 객체의 __iter__ 메서드 호출 -> 반복가능한 객체에서 이터레이트를 발환 
* next : __next__ 메서드 호출 -> 이터레이터에서 값을 차례대로 꺼냄
"""
next(iter(train_iterator)).text.shape # 문장의 길이(text 최대 크기) * 배치 사이즈 

torch.Size([73, 64])

# 모델 생성하기
> 바닐라 RNN을 이용하여 간단한 모델을 생성. 모델의 각 input/output 벡터의 사이즈는 다음과 같음
* text : [sentence length, batch size]
* embedded : [sentence length, batch_size, embedding dim]
* output : [sentence length, batch size, hidden dim]
* hidden : [1, batch size, hidden dim]
# https://justkode.kr/deep-learning/pytorch-rnn</br>


> [Q1] input data = sequence length </br>
  input data shape = ( batch size, sequence length, dimension) 
  


> [Q2] Dimension의 개념.. 이해한 것 설명해보기 </br> 
   ex) embedding dimension, hidden dimension </br>

   """</br>
   "차원 or피처의 개수"</br> 
   1차원 직선, 2차원 평면과 같이 x, y 축으로 나타내는 '축' = dimension </br>
   (ML) 데이터를 표현하는데 필요한 축 'dimension' => feature의 개수 = dimension </br> ex) 정규분표의 경우, 평균, 표준편차 두 변수가 있음. feature 개수 2개 = 2-Dimension임 

   """</br>

> [Q3] hidden dimension의 개념? 작동방법? 
   hidden_state = output data로 몇 차원의 vector를 출력받을지 결정 
   hidden_size = 2 = output_size( , , 2) 
   HOW? 출력 직전에 output size, hidden size로 나뉘게됨 

In [ ]:
import torch.nn as nn

In [ ]:
class RNN(nn.Module):

  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  
  def forward(self, text):
    # text : [sent_len, batch_size]
    embedded = self.embedding(text)
    # embedded : [sent_len, batch_size, emb_dim]
    output, hidden = self.rnn(embedded)
    # output : [sent_len, batch_size, hidden_dim]
    # hidden : [1, batch_size, hidden_dim]
    assert torch.equal(output[-1,:,:], hidden.squeeze(0))

    return self.fc(hidden.squeeze(0)) # [batch_size, output_dim]

### 모델의 하이퍼파라미터를 다음과 같이 설정한 후 불러오기

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

### 이 모델의 파라미터 수를 다음과 같이 추출 

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,} 개의 파라미터를 가지고 있습니다.')

이 모델은 2,592,105 개의 파라미터를 가지고 있습니다.


# 모델 훈련
> SGD optimizer를 이용하여 모델 훈련

[Q1]  Optimizer의 개념 = parameter 어떻게? 업데이트 할 것인가 ex. SGD, Adam etc 

</br> 손실함수(Loss function)와 다른 개념?</br> 
[Q2] 손실함수는 forward-backward하면서loss를 계산하는 방식?

> Optimizer은 backward할 때 어떻게 각각의 파라미터(w, b)를 업데이트 해줄 것인가?

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

### 손실함수는 bianry cross entropy with logits
> 임의의 실수를 입력으로 받아서 sigmoid 함수를 취해 0과 1 사이의 값으로 변환한 뒤 label과의 binary cross entropy를 계산함

In [ ]:
criterion = nn.BCEWithLogitsLoss()

### 모델과 손실함수를 GPU에 올리자

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

### 이제 평가를 위해 임의의 실수를 0과 1 두 정수 중 하나로 변환하는 함수를 만들자

In [ ]:
def binary_accuracy(preds, y):
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)
  return acc

# 모델의 훈련과 평가를 위한 함수를 만들기 
> model의 output size는 [batch_size, output_dim]임. output_dim=1이므로 이 차원을 없애줘야 label과 비교할 수 있어 squeeze(1)를 적용함


> [Q] 루프 = Iterator 

In [ ]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.train() 

  """
  * optimizer.zero_grad()
  루프가 한번 돌고나서 역전파 하기 전에 반드시 zero_grad()로 .grad값들을 0으로 초기화시킨 후 학습 진행해야함 
  why? 안그러면 이전 루프에 저장된 grad가 다음 루프의 업뎃에도 간섭하게됨 

  """

  for batch in iterator:
    optimizer.zero_grad()
    predictions = model(batch.text).squeeze(1)
    loss = criterion(predictions, batch.label)
    acc = binary_accuracy(predictions, batch.label)
  
  """
  한 루프에서 업뎃 위해 loss.backward() 호출 -> 각 파라미터들의 grad값에 변화 
  """
    loss.backward()

  """
  backward하며 미분 -> 손실함수에 끼친 영향력(변화량) 구하고 optimizer step을 통해 손실함수를 최적화하도록 파라미터를 업데이트함 
  """
    optimizer.step()

    epoch_loss += loss.item() # tensor에 item()을 취하면 value를 반환함
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)


### 평가를 위한 함수는 그래디언트 업데이트를 하지 않아야 하므로 with torch.no_grad(): 구문으로 감싸도록 함

> [Q] overftting의 기준? 예측값과 실제값의 정확도가 5% 이상 차이나면 보통 오버피팅이라고하는지..? 


In [ ]:
def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0
  
  """
  model 학습 후 모델 평가할 때 model.eval 입력하고 시작 -> model이 학습을 끝낸 후, model.eval()을 통해 평가모드로 바꾸겠다는 의미
  Why? 훈련모드와 평가모드는 구조가 다르기 때문에- 평가모드엔 다른른 layer가 있기 때문에 
  """
  model.eval() # 이 모델은 상관없음

  with torch.no_grad():
    for batch in iterator:
      predictions = model(batch.text).squeeze(1)
      loss = criterion(predictions, batch.label)
      acc = binary_accuracy(predictions, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss/len(iterator), epoch_acc/len(iterator)
      

SyntaxError: ignored

### epoch마다 걸린 훈련시간을 측정하는 함수 

In [ ]:
import time

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

# 실제 훈련시키기 * 검증셋의 손실이 개선되면 모델을 저장하도록 구현 

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

In [ ]:
for epoch in range(N_EPOCHS):
  start_time = time.time()

  train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss: 
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'tutl-model.pt')

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\t Train Loss: {train_loss:.3f} | Train Acc : {train_acc*100:0.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} | Val Acc: {valid_acc*100:0.2f}% ')

Epoch: 01 | Epoch Time: 4m 56s
	 Train Loss: 0.693 | Train Acc : 49.79%
	 Val. Loss: 0.695 | Val Acc: 50.89% 
Epoch: 02 | Epoch Time: 4m 31s
	 Train Loss: 0.693 | Train Acc : 50.18%
	 Val. Loss: 0.694 | Val Acc: 51.17% 
Epoch: 03 | Epoch Time: 4m 30s
	 Train Loss: 0.693 | Train Acc : 50.13%
	 Val. Loss: 0.693 | Val Acc: 51.51% 
Epoch: 04 | Epoch Time: 4m 31s
	 Train Loss: 0.693 | Train Acc : 49.80%
	 Val. Loss: 0.693 | Val Acc: 52.38% 
Epoch: 05 | Epoch Time: 4m 28s
	 Train Loss: 0.693 | Train Acc : 50.18%
	 Val. Loss: 0.692 | Val Acc: 52.92% 


###  모델의 성능이 좋지 않아 훈련이 거의 되지 않음을 확인할 수 있었음 

# 검증 셋에서 가장 좋은 결과를 받은 모델 기준으로 테스트셋의 결과를 측정해보기

In [ ]:
model.load_state_dict(torch.load('tutl-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc : {test_acc*100:.2f}%')

Test Loss: 0.692 | Test Acc : 53.06%
